In [1]:
%%configure -f 
{"conf":{
     "spark.app.name":"Bhavesh_Relevancy_notebook", "spark.yarn.queue": "default", "spark.jars": "/apps/Jars/obelisk-retail-legos.jar"}
}

In [2]:
import ai.couture.obelisk.commons.Constants.{DB_PRODUCT_INTERACTIONS_PATH, PROCESSED_GA_DATA, GA_USER_CLICK_DATA}
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io.{DFToParquet, HdfsUtils, ParquetToDF, DFToCSV, CSVToDF}
import ai.couture.obelisk.commons.utils.DateTimeUtil.{getFutureDateFromHere, getNumberOfDaysBetweenTwoDates, getOldDateFromHere, getOldDate}
import ai.couture.obelisk.retail.legos.BaseBlocksRevised
import ai.couture.obelisk.retail.legos.HyperParams.PLPRecommendationConstants._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.DoubleType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9756,application_1672415114980_0753,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import ai.couture.obelisk.commons.Constants.{DB_PRODUCT_INTERACTIONS_PATH, PROCESSED_GA_DATA, GA_USER_CLICK_DATA}
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io.{DFToParquet, HdfsUtils, ParquetToDF, DFToCSV, CSVToDF}
import ai.couture.obelisk.commons.utils.DateTimeUtil.{getFutureDateFromHere, getNumberOfDaysBetweenTwoDates, getOldDateFromHere, getOldDate}
import ai.couture.obelisk.retail.legos.BaseBlocksRevised
import ai.couture.obelisk.retail.legos.HyperParams.PLPRecommendationConstants._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.DoubleType


In [3]:
var interactionsDB = ParquetToDF.getDF(
    DB_PRODUCT_INTERACTIONS_PATH).filter(
        col("event") === "Checkout" && col("purchase") === "New" && col(PRODUCTID).isNotNull && col(USERID).isNotNull && col(QUANTITY) > 0 && col("booked_rev")>0
    )
interactionsDB.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

interactionsDB: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string, productsku: string ... 28 more fields]
root
 |-- productid: string (nullable = true)
 |-- productsku: string (nullable = true)
 |-- userid: long (nullable = true)
 |-- interactionid: string (nullable = true)
 |-- baseprice: double (nullable = true)
 |-- cancelledquantity: long (nullable = true)
 |-- mrp: double (nullable = true)
 |-- quantity: long (nullable = true)
 |-- returnwindow: long (nullable = true)
 |-- price: double (nullable = true)
 |-- isluxe: long (nullable = true)
 |-- status: long (nullable = true)
 |-- store: long (nullable = true)
 |-- ordertotalprice: double (nullable = true)
 |-- pincode: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- event: string (nullable = true)
 |-- penaltycharge: long (nullable = true)
 |-- orderid: string (nullable = true)
 |-- orderquantity: long (nullable = true)
 |-- refundamount: double (nullable = true)
 |-- booked_rev: do

In [4]:
var gaData = ParquetToDF.getDF("/data/ecomm/ajio/processed/GaCohortVisibilityLegosFNL")
gaData.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gaData: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 6 more fields]
root
 |-- productid: string (nullable = true)
 |-- clusterid: string (nullable = true)
 |-- PDPCountPerDay: double (nullable = true)
 |-- PLPViewsPerDay: double (nullable = true)
 |-- PLPClicksPerDay: double (nullable = true)
 |-- TotalAddToCartPerDay: double (nullable = true)
 |-- CheckoutsPerDay: double (nullable = true)
 |-- date: date (nullable = true)



In [5]:
var userClusterMapping = ParquetToDF.getDF("/data/ecomm/ajio/processed/segmentationFiles/PremiumClustering/userClusters")select(USERID, CLUSTERID)
userClusterMapping.printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

userClusterMapping: org.apache.spark.sql.DataFrame = [userid: string, clusterid: string]
root
 |-- userid: string (nullable = true)
 |-- clusterid: string (nullable = true)



In [6]:
var experimentDate: String = "2022-12-06"
interactionsDB = interactionsDB.filter(col(DATE) > getOldDateFromHere(30, experimentDate, "yyyy-MM-dd") && col(DATE) <= experimentDate)
gaData = gaData.filter(col(DATE) > getOldDateFromHere(30, experimentDate, "yyyy-MM-dd") && col(DATE) <= experimentDate)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

experimentDate: String = 2022-12-06
interactionsDB: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string, productsku: string ... 28 more fields]
gaData: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 6 more fields]


In [14]:
interactionsDB = interactionsDB.join(userClusterMapping, Seq(USERID)).select(PRODUCTID, INTERACTIONID, QUANTITY, "booked_rev", DATE, CLUSTERID)
// interactionsDB.show(5, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

interactionsDB: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string, interactionid: string ... 4 more fields]


In [15]:
var df = interactionsDB.groupBy(PRODUCTID, CLUSTERID, DATE).agg(sum("booked_rev") as "Revenue", sum(QUANTITY) as "quantity", countDistinct(INTERACTIONID) as "sales")
df.show(5, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 4 more fields]
+------------------+---------+----------+--------+--------+-----+
|productid         |clusterid|date      |Revenue |quantity|sales|
+------------------+---------+----------+--------+--------+-----+
|462566319_navy    |cluster0 |2022-11-08|118560.0|156     |154  |
|469251942_darknavy|cluster0 |2022-11-20|543.23  |1       |1    |
|6005533020_multi  |cluster0 |2022-11-15|13056.0 |51      |50   |
|441134776_jetblack|cluster0 |2022-11-21|12188.3 |20      |20   |
|469254665_black   |cluster0 |2022-11-28|30480.43|24      |24   |
+------------------+---------+----------+--------+--------+-----+
only showing top 5 rows



In [17]:
df = df.join(gaData, Seq(PRODUCTID, CLUSTERID, DATE))
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 9 more fields]
root
 |-- productid: string (nullable = true)
 |-- clusterid: string (nullable = true)
 |-- date: date (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- quantity: long (nullable = true)
 |-- sales: long (nullable = false)
 |-- PDPCountPerDay: double (nullable = true)
 |-- PLPViewsPerDay: double (nullable = true)
 |-- PLPClicksPerDay: double (nullable = true)
 |-- TotalAddToCartPerDay: double (nullable = true)
 |-- CheckoutsPerDay: double (nullable = true)



In [19]:
df = df.select(PRODUCTID, CLUSTERID, DATE, QUANTITY, "sales", "Revenue", "PLPViewsPerDay")
df.select(max(DATE), min(DATE)).head(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 5 more fields]
res16: Array[org.apache.spark.sql.Row] = Array([2022-12-06,2022-11-07])


In [20]:
DFToCSV.putDF("/data/bhavesh/CSV Files/newTargetAnalysisDF.csv", df.filter(col(CLUSTERID) === "cluster1").repartition(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
var mle = df.filter(col(DATE) === experimentDate).select(PRODUCTID, CLUSTERID, "Revenue", "sales", "PLPViewsPerDay")
var e_theta = df.filter(col(DATE) < experimentDate).select(PRODUCTID, CLUSTERID, "Revenue", "sales", "PLPViewsPerDay")
mle = mle.groupBy(PRODUCTID, CLUSTERID).agg(sum("Revenue") as "revenue", sum("sales") as "sales", sum("PLPViewsPerDay") as "PLPViewsPerDay")
e_theta = e_theta.groupBy(PRODUCTID, CLUSTERID).agg(sum("Revenue") as "revenue", sum("sales") as "sales", sum("PLPViewsPerDay") as "PLPViewsPerDay")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mle: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 3 more fields]
e_theta: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 3 more fields]
mle: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 3 more fields]
e_theta: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 3 more fields]


In [23]:
mle = mle.withColumn("revenue_per_product_plp_views", when(col("PLPViewsPerDay")>0, col("revenue")/col("PLPViewsPerDay")).otherwise(lit(0.0)))
mle = mle.withColumn("sales_per_product_plp_views", when(col("PLPViewsPerDay")>0, col("sales")/col("PLPViewsPerDay")).otherwise(lit(0.0)))

e_theta = e_theta.withColumn("revenue_per_product_plp_views", when(col("PLPViewsPerDay")>0, col("revenue")/col("PLPViewsPerDay")).otherwise(lit(0.0)))
e_theta = e_theta.withColumn("sales_per_product_plp_views", when(col("PLPViewsPerDay")>0, col("sales")/col("PLPViewsPerDay")).otherwise(lit(0.0)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mle: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 4 more fields]
mle: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 5 more fields]
e_theta: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 4 more fields]
e_theta: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 5 more fields]


In [67]:
var final_df = mle.join(e_theta.withColumnRenamed("revenue_per_product_plp_views", "revenue_per_product_plp_views_e_theta").withColumnRenamed("sales_per_product_plp_views", "sales_per_product_plp_views_e_theta").select(PRODUCTID, CLUSTERID, "revenue_per_product_plp_views_e_theta", "sales_per_product_plp_views_e_theta"), Seq(PRODUCTID, CLUSTERID), "left_outer").na.fill(0.0, Seq("revenue_per_product_plp_views_e_theta", "sales_per_product_plp_views_e_theta"))
final_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

final_df: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 7 more fields]
root
 |-- productid: string (nullable = true)
 |-- clusterid: string (nullable = true)
 |-- revenue: double (nullable = true)
 |-- sales: integer (nullable = true)
 |-- PLPViewsPerDay: double (nullable = true)
 |-- revenue_per_product_plp_views: double (nullable = true)
 |-- sales_per_product_plp_views: double (nullable = true)
 |-- revenue_per_product_plp_views_e_theta: double (nullable = false)
 |-- sales_per_product_plp_views_e_theta: double (nullable = false)



In [68]:
final_df = final_df.withColumn("BayesianParameterOnRevenuePerView", (col("revenue_per_product_plp_views_e_theta")/(lit(1.0)+col("PLPViewsPerDay"))) + ((col("PLPViewsPerDay")*col("revenue_per_product_plp_views"))/(lit(1.0)+col("PLPViewsPerDay"))))
final_df = final_df.withColumn("BayesianParameterOnSalesPerView", (col("sales_per_product_plp_views_e_theta")/(lit(1.0)+col("PLPViewsPerDay"))) + ((col("PLPViewsPerDay")*col("sales_per_product_plp_views"))/(lit(1.0)+col("PLPViewsPerDay"))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

final_df: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 8 more fields]
final_df: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 9 more fields]


In [72]:
DFToCSV.putDF("/data/bhavesh/CSV Files/BayesianTarget.csv", final_df.filter(col(CLUSTERID) === "cluster1").repartition(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
DFToCSV.putDF("/data/bhavesh/CSV Files/ExperimentDayTargetVariable.csv",mle.filter(col(CLUSTERID) === "cluster1").repartition(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
DFToCSV.putDF("/data/bhavesh/CSV Files/HistoryDaysTargetVariable.csv",e_theta.filter(col(CLUSTERID) === "cluster1").repartition(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
var final_df = CSVToDF.getDF("/data/bhavesh/CSV Files/BayesianTarget.csv", inferSchema=true)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

final_df: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 9 more fields]


In [4]:
var productAttributes = (ParquetToDF.getDF(PRODUCTS_LEGOS_PATH)
.withColumnRenamed(ITEM_ID,PRODUCTID)
.withColumnRenamed(SIMILAR_GROUP_LEVEL,BRICKID)
.select(PRODUCTID,BRICKID).distinct())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productAttributes: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string, brickid: string]


In [5]:
var brickDetails = (ParquetToDF.getDF(BRICK_DETAILS_LEGOS_PATH)
.select(col(L1NAME), col(L2NAME), col(BRICKNAME), col(SIMILAR_GROUP_LEVEL).as(BRICKID)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

brickDetails: org.apache.spark.sql.DataFrame = [l1name: string, l2name: string ... 2 more fields]


In [6]:
final_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- productid: string (nullable = true)
 |-- clusterid: string (nullable = true)
 |-- revenue: double (nullable = true)
 |-- sales: integer (nullable = true)
 |-- PLPViewsPerDay: double (nullable = true)
 |-- revenue_per_product_plp_views: double (nullable = true)
 |-- sales_per_product_plp_views: double (nullable = true)
 |-- revenue_per_product_plp_views_e_theta: double (nullable = true)
 |-- sales_per_product_plp_views_e_theta: double (nullable = true)
 |-- BayesianParameterOnRevenuePerView: double (nullable = true)
 |-- BayesianParameterOnSalesPerView: double (nullable = true)



In [7]:
var m: Double = final_df.agg(callUDF("percentile_approx", col("PLPViewsPerDay"), lit(0.9))).head().get(0) match { case d: Double => d}
var C_revenue_per_view: Double = final_df.agg(mean("revenue_per_product_plp_views")).head().get(0) match { case d: Double => d}
var C_sales_per_view: Double = final_df.agg(mean("sales_per_product_plp_views")).head().get(0) match { case d: Double => d}
println(m, C_revenue_per_view, C_sales_per_view)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

m: Double = 1325.0
C_revenue_per_view: Double = 25.90462420368837
C_sales_per_view: Double = 0.02539355319889995
(1325.0,25.90462420368837,0.02539355319889995)


In [8]:
final_df = (
    final_df
    .withColumn("NormalizedRevenuePerViews", ((col("revenue_per_product_plp_views")*col("PLPViewsPerDay")) + lit(m*C_revenue_per_view))/(col("PLPViewsPerDay")+lit(m)))
    .withColumn("NormalizedSalesPerViews", ((col("sales_per_product_plp_views")*col("PLPViewsPerDay")) + lit(m*C_sales_per_view))/(col("PLPViewsPerDay")+lit(m)))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

final_df: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 11 more fields]


In [9]:
final_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- productid: string (nullable = true)
 |-- clusterid: string (nullable = true)
 |-- revenue: double (nullable = true)
 |-- sales: integer (nullable = true)
 |-- PLPViewsPerDay: double (nullable = true)
 |-- revenue_per_product_plp_views: double (nullable = true)
 |-- sales_per_product_plp_views: double (nullable = true)
 |-- revenue_per_product_plp_views_e_theta: double (nullable = true)
 |-- sales_per_product_plp_views_e_theta: double (nullable = true)
 |-- BayesianParameterOnRevenuePerView: double (nullable = true)
 |-- BayesianParameterOnSalesPerView: double (nullable = true)
 |-- NormalizedRevenuePerViews: double (nullable = true)
 |-- NormalizedSalesPerViews: double (nullable = true)



In [10]:
final_df = (
    final_df
    .withColumn("RankByRevenuePerViews", row_number().over(Window.partitionBy(CLUSTERID).orderBy(desc("revenue_per_product_plp_views"))))
    .withColumn("RankBySalesPerViews", row_number().over(Window.partitionBy(CLUSTERID).orderBy(desc("sales_per_product_plp_views"))))
    .withColumn("RankByBayesianRevenuePerViews", row_number().over(Window.partitionBy(CLUSTERID).orderBy(desc("BayesianParameterOnRevenuePerView"))))
    .withColumn("RankByBayesianSalesPerViews", row_number().over(Window.partitionBy(CLUSTERID).orderBy(desc("BayesianParameterOnSalesPerView"))))
    .withColumn("RankByNormalizedRevenuePerViews", row_number().over(Window.partitionBy(CLUSTERID).orderBy(desc("NormalizedRevenuePerViews"))))
    .withColumn("RankByNormalizedSalesPerViews", row_number().over(Window.partitionBy(CLUSTERID).orderBy(desc("NormalizedSalesPerViews"))))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

final_df: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 17 more fields]


In [11]:
final_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- productid: string (nullable = true)
 |-- clusterid: string (nullable = true)
 |-- revenue: double (nullable = true)
 |-- sales: integer (nullable = true)
 |-- PLPViewsPerDay: double (nullable = true)
 |-- revenue_per_product_plp_views: double (nullable = true)
 |-- sales_per_product_plp_views: double (nullable = true)
 |-- revenue_per_product_plp_views_e_theta: double (nullable = true)
 |-- sales_per_product_plp_views_e_theta: double (nullable = true)
 |-- BayesianParameterOnRevenuePerView: double (nullable = true)
 |-- BayesianParameterOnSalesPerView: double (nullable = true)
 |-- NormalizedRevenuePerViews: double (nullable = true)
 |-- NormalizedSalesPerViews: double (nullable = true)
 |-- RankByRevenuePerViews: integer (nullable = true)
 |-- RankBySalesPerViews: integer (nullable = true)
 |-- RankByBayesianRevenuePerViews: integer (nullable = true)
 |-- RankByBayesianSalesPerViews: integer (nullable = true)
 |-- RankByNormalizedRevenuePerViews: integer (nullable = true)
 |

In [12]:
DFToCSV.putDF("/data/bhavesh/CSV Files/TargetVariableRanked.csv", final_df.repartition(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
final_df = CSVToDF.getDF("/data/bhavesh/CSV Files/TargetVariableRanked.csv", inferSchema=true)
final_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

final_df: org.apache.spark.sql.DataFrame = [productid: string, clusterid: string ... 17 more fields]
root
 |-- productid: string (nullable = true)
 |-- clusterid: string (nullable = true)
 |-- revenue: double (nullable = true)
 |-- sales: integer (nullable = true)
 |-- PLPViewsPerDay: double (nullable = true)
 |-- revenue_per_product_plp_views: double (nullable = true)
 |-- sales_per_product_plp_views: double (nullable = true)
 |-- revenue_per_product_plp_views_e_theta: double (nullable = true)
 |-- sales_per_product_plp_views_e_theta: double (nullable = true)
 |-- BayesianParameterOnRevenuePerView: double (nullable = true)
 |-- BayesianParameterOnSalesPerView: double (nullable = true)
 |-- NormalizedRevenuePerViews: double (nullable = true)
 |-- NormalizedSalesPerViews: double (nullable = true)
 |-- RankByRevenuePerViews: integer (nullable = true)
 |-- RankBySalesPerViews: integer (nullable = true)
 |-- RankByBayesianRevenuePerViews: integer (nullable = true)
 |-- RankByBayesianSalesP

In [14]:
val Ranks = Seq(100, 1000, 5000, 10000)
val PLPLevelNames=Seq(L1NAME,L2NAME,BRICKNAME)
val RankColumn = Seq("RankByRevenuePerViews", "RankBySalesPerViews", "RankByBayesianRevenuePerViews", "RankByBayesianSalesPerViews", "RankByNormalizedRevenuePerViews", "RankByNormalizedSalesPerViews")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Ranks: Seq[Int] = List(100, 1000, 5000, 10000)
PLPLevelNames: Seq[String] = List(l1name, l2name, brickname)
RankColumn: Seq[String] = List(RankByRevenuePerViews, RankBySalesPerViews, RankByBayesianRevenuePerViews, RankByBayesianSalesPerViews, RankByNormalizedRevenuePerViews, RankByNormalizedSalesPerViews)


In [15]:
var rankDF: DataFrame = null
var rankDistributionDFForAllRankColumns: DataFrame = null

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rankDF: org.apache.spark.sql.DataFrame = null
rankDistributionDFForAllRankColumns: org.apache.spark.sql.DataFrame = null


In [17]:
var productBrandMapping = (
    ParquetToDF.getDF(PRODUCTS_LEGOS_PATH)
        .selectExpr(s"$ITEM_ID as $PRODUCTID", s"$BRANDNAME")
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), "&", " "))
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), "_", " "))
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), "\\.", " "))
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), "@", " "))
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), " +", " ")) //replace multiple spaces with a singe space
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), " ", "-"))
        .distinct()
        .na.drop
)

var premiumBrands = (
    CSVToDF.getDF("/data/ecomm/ajio/metadata/premiumbrands.csv")
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), "&", " "))
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), "_", " "))
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), "\\.", " "))
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), "@", " "))
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), " +", " ")) //replace multiple spaces with a singe space
        .withColumn(BRANDNAME, regexp_replace(lower(col(BRANDNAME)), " ", "-"))
        /*
        //We deliberately avoid substitutions like these in code. We prefer to edit the csv itself.
        //If there is an issue with the substition, it is A LOT easier to submit a corrected csv than it is to redeploy a jar
        .withColumn(BRANDNAME,
          when(col(BRANDNAME)==="accessorize","accessorize-london")
            .otherwise(col(BRANDNAME))
        )*/ .distinct()
        .withColumn("isPremiumBrand", lit(1))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productBrandMapping: org.apache.spark.sql.DataFrame = [productid: string, brandname: string]
premiumBrands: org.apache.spark.sql.DataFrame = [brandname: string, isPremiumBrand: int]


In [ ]:
for(plpLevelName <- PLPLevelNames){
    for(rank <- Ranks){
        rankDistributionDFForAllRankColumns = null
        for(rankColumn <- RankColumn){
            rankDF = (
                final_df
                .filter(col(rankColumn) <= rank && col(rankColumn) > 0)
                .join(productAttributes.select(PRODUCTID, BRICKID), Seq(PRODUCTID))
                .join(brickDetails, Seq(BRICKID), "left")
                .groupBy(CLUSTERID,plpLevelName)
                .agg(countDistinct(PRODUCTID).alias("ProductCount"))
            )
            if (rankDistributionDFForAllRankColumns == null) {
                  rankDistributionDFForAllRankColumns = rankDF.withColumnRenamed("ProductCount", s"ProductCount$rankColumn")
            } else {
                  rankDistributionDFForAllRankColumns = (
                      rankDistributionDFForAllRankColumns
                      .join(
                        rankDF.withColumnRenamed("ProductCount", s"ProductCount$rankColumn"),
                        Seq(CLUSTERID, plpLevelName), "full_outer"
                      )
                  )
            }
        }
        rankDistributionDFForAllRankColumns = rankDistributionDFForAllRankColumns.na.fill(0)
        DFToCSV.putDF(s"/data/bhavesh/CSV Files/PLPResultsEvaluation/2022-12-06/RankingComparisonMetrics/RankwiseProductDistributionDF/$plpLevelName/rankLimit=$rank", rankDistributionDFForAllRankColumns.repartition(1), hasHeader = true)
    }
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
var brandDistributionDFForAllRankColumns: DataFrame = null

for(rank <- Ranks){
    brandDistributionDFForAllRankColumns = null
    for(rankColumn <- RankColumn){
        rankDF = (
            final_df
            .filter(col(rankColumn) <= rank && col(rankColumn) > 0)
            .join(productBrandMapping, Seq(PRODUCTID), "left")
            .groupBy(CLUSTERID,BRANDNAME)
            .agg(countDistinct(PRODUCTID).alias("ProductCount"))
        )
        if (brandDistributionDFForAllRankColumns == null) {
              brandDistributionDFForAllRankColumns = rankDF.withColumnRenamed("ProductCount", s"ProductCount$rankColumn")
        } else {
              brandDistributionDFForAllRankColumns = (
                  brandDistributionDFForAllRankColumns
                  .join(
                    rankDF.withColumnRenamed("ProductCount", s"ProductCount$rankColumn"),
                    Seq(CLUSTERID, BRANDNAME), "full_outer"
                  )
              )
        }
    }
    brandDistributionDFForAllRankColumns = brandDistributionDFForAllRankColumns.na.fill(0).join(premiumBrands, Seq(BRANDNAME), "left").na.fill(0, Seq("isPremiumBrand"))
    brandDistributionDFForAllRankColumns = brandDistributionDFForAllRankColumns.select(CLUSTERID,BRANDNAME,"isPremiumBrand", "ProductCountRankByRevenuePerViews", "ProductCountRankBySalesPerViews", "ProductCountRankByBayesianRevenuePerViews", "ProductCountRankByBayesianSalesPerViews", "ProductCountRankByNormalizedRevenuePerViews", "ProductCountRankByNormalizedSalesPerViews")
    DFToCSV.putDF(s"/data/bhavesh/CSV Files/PLPResultsEvaluation/2022-12-06/RankingComparisonMetrics/RankwiseBrandDistributionDF/rankLimit=$rank", brandDistributionDFForAllRankColumns.repartition(1), hasHeader = true)
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 